In [1]:
import re
import os
import json
import numpy as np
from tqdm import tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from utils.encode import GenericPLMEncoder

# Create the elastic instance
elastic = Elasticsearch(
    "http://localhost:9200",
    request_timeout=1000000
)

# Successful response!
elastic.info()

ObjectApiResponse({'name': 'ubuntu', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'G7JzeSt3Q5OoTS6zBdAaWQ', 'version': {'number': '8.4.1', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '2bd229c8e56650b42e40992322a76e7914258f0c', 'build_date': '2022-08-26T12:11:43.232597118Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
emb = np.random.rand(768).astype(np.float32)

elastic.search(
    index="wenshu",
    size=2,
    from_=0,
    _source=False,
    fields=["case_name", "content", {"field": "publish_date", "format": "year_month_day"}, "court_name", "case_type"],
    query={
        # "combined_fields": {
        #     "query": "测试",
        #     "fields": ["case_name", "content"]
        # },
        # "match_phrase": {
        #     "case_name": "彭绍波与陈忠远民间借贷纠纷一审民事判决书",
        # },
        "multi_match" : {
            "query": "打了一个借条",
            "type": "phrase",
            "fields": ["case_name", "content"]
        }
    },
    aggs={
        "agg-court": {
            "terms": {
                "field": "court_name"
            }
        }
    },
    # knn={
    #     "field": "vector",
    #     "query_vector": (emb / np.linalg.norm(emb)).tolist(),
    #     "k": 10,
    #     "num_candidates": 10,
    #     # "boost": 1
    # },
    # highlight={
    #     "fields": {
    #         "content": {
    #             "pre_tags" : ["<strong>"],
    #             "post_tags": ["</strong>"],
    #             "number_of_fragments": 1,
    #         }
    #     }
    # },
    # post_filter={
    #     "bool": {
    #         "filter": [
    #             {
    #                 "terms": {"court_name": ["上海市长宁区人民法院"]}
    #             }
    #         ]
    #     }
    # }
)["hits"]

In [ ]:
file = "p2-1.filtered"
model = "DPR"

embeddings = np.memmap(
    os.path.join("data/encode", model, "wenshu", file, "text_embeddings.mmp"),
    dtype=np.float32,
    mode="r"
).reshape(-1, 768)

def gendata():
    with open(f"../../../Data/wenshu/{file}.txt", encoding="utf-8") as f:
        for i, line in enumerate(f):
            case = json.loads(line.strip())
            del case["crawl_time"]
            del case["legal_base"]
            del case["tf_content"]
            del case["id"]
            case["vector"] = embeddings[i].tolist()
            yield case

for x in tqdm(gendata(), desc="Indexing", total=embeddings.shape[0]):
    elastic.index(index="wenshu", document=x)

In [4]:
elastic.indices.create(
    index="wenshu",
    settings={
        'analysis': {
            'analyzer': {
                # we must set the default analyzer
                "default": {
                    "type": "smartcn"
                }
            }
        },
        "index.mapping.ignore_malformed": True
    },
    mappings={
        "properties": {
            # field name
            "doc_id": {
                "type": "keyword",
            },
            "court_name": {
                "type": "keyword",
            },
            "court_id": {
                "type": "keyword",
            },
            "court_province": {
                "type": "keyword",
            },
            "court_city": {
                "type": "keyword",
            },
            "court_region": {
                "type": "keyword",
            },
            "court_district": {
                "type": "keyword"
            },
            "pub_prosecution_org": {
                "type": "keyword"
            },
            "case_type": {
                "type": "keyword",
            },
            "cause": {
                "type": "keyword",
            },
            "trial_round": {
                "type": "keyword"
            },
            "content": {
                "type": "text"
            },
            "appellor": {
                "type": "keyword"
            },
            "vector": {
                "type": "dense_vector",
                "dims": 768,
                # enable hnsw
                "index": True,
                # inner product only allows unit-length vector
                "similarity": "dot_product"  
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'wenshu'})

In [ ]:
elastic.indices.get_mapping(index="wenshu")

In [3]:
# delete index
elastic.indices.delete(index="wenshu")

ObjectApiResponse({'acknowledged': True})

In [4]:
elastic.indices.get_alias(index="*")

/tmp/ipykernel_113771/2122939163.py:1: ElasticsearchWarning: this request accesses system indices: [.security-7], but in a future major version, direct access to system indices will be prevented by default
  elastic.indices.get_alias(index="*")


ObjectApiResponse({'.security-7': {'aliases': {'.security': {'is_hidden': True}}}})

In [2]:
elastic.delete_by_query(
    index="wenshu", 
    query={
        "match_all": {}
    }
)

ConflictError: ConflictError(409, "{'took': 983, 'timed_out': False, 'total': 32001, 'deleted': 0, 'batches': 1, 'version_conflicts': 1000, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': [{'index': 'wenshu', 'id': 'I3y2SoMBK2v3rVegNbOj', 'cause': {'type': 'version_conflict_engine_exception', 'reason': '[I3y2SoMBK2v3rVegNbOj]: version conflict, required seqNo [289854], primary term [1]. but no document was found', 'index_uuid': 'tq-oXMnySiOWArn6Og-3jw', 'shard': '0', 'index': 'wenshu'}, 'status': 409}, {'index': 'wenshu', 'id': 'JHy2SoMBK2v3rVegNbOp', 'cause': {'type': 'version_conflict_engine_exception', 'reason': '[JHy2SoMBK2v3rVegNbOp]: version conflict, required seqNo [289855], primary term [1]. but no document was found', 'index_uuid': 'tq-oXMnySiOWArn6Og-3jw', 'shard': '0', 'index': 'wenshu'}, 'status': 409}, {'index': 'wenshu', 'id': 'JXy2SoMBK2v3rVegNbO1', 'cause': {'type': 'version_co

In [ ]:
elastic.search(
    index="wenshu",
    query={
        "match_all": {}
    },
    size=1,
    sort=[
        {
        "_timestamp": {
            "order": "desc"
        }
        }
    ]
)["hits"]

In [2]:
elastic.count(
    index="wenshu",
    query={
        "match_all": {
        }
    }
)

ObjectApiResponse({'count': 300001, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [ ]:
skipped = 0
file = "p5"
with open(f"../../../Data/wenshu/{file}.txt", encoding="utf-8") as f, open(f"../../../Data/wenshu/{file}.filtered.txt", "w", encoding="utf-8") as g:
    for i, line in enumerate(tqdm(f)):
        fields = json.loads(line.strip())

        if fields["basics_text"]:
            content = fields["basics_text"].replace("/n", "")
        else:
            content = fields["content"]
            if not content:
                skipped += 1
                continue
            else:
                content = content.replace("\n", "")
                for x in ("指控：", "指控，", "查明：", "查明，", "诉称：", "诉称，", "请求：", "理由：", "认定：", "认定:", "认定，", "查认为，"):
                    idx = content.rfind(x)
                    if idx != -1:
                        break
                if idx == -1:
                    skipped += 1
                    # print(content)
                    # if skipped > 10:
                    #     break
                    continue
                else:
                    content = content[idx:]
        # write filter content to a new field
        fields["tf_content"] = content
        g.write(json.dumps(fields, ensure_ascii=False) + "\n")

skipped